In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import random
import gc
import os
import datetime
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
        
path_data = "../input/"
path_train = path_data + "train.csv"
path_test = path_data + "test.csv"
path_building = path_data + "building_metadata.csv"
path_weather_train = path_data + "weather_train.csv"
path_weather_test = path_data + "weather_test.csv"

plt.style.use("seaborn")
sns.set(font_scale=1)

myfavouritenumber = 0
seed = myfavouritenumber
random.seed(seed)

pd.set_option('max_rows', 9999)
pd.set_option('max_columns', 9999)

today = datetime.datetime.now().strftime('%Y%m%d')

/Users/imamuramasataka/Documents/kaggle/ashrae-energy-prediction/.venv/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
## Memory optimization

# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16

from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
def extract_id_meter(df, building_id, meter):
    temp = df[df['building_id'] == building_id].copy()
    temp = temp[temp['meter'] == meter]
    return temp

In [8]:
def prep_weather_data(df, mode='train'):
    
    # relative Hummd  #####################################################################
    # https://soudan1.biglobe.ne.jp/qa5356721.html
    a_temp = df['air_temperature'].values
    d_temp = df['dew_temperature'].values

    def SaturatedWaterVaporPressure(values):
        return 6.11 * 10 ** (7.5 * values / (237.3 + values))

    a_temp = SaturatedWaterVaporPressure(a_temp)
    d_temp = SaturatedWaterVaporPressure(d_temp)

    df['relative_hummd'] = d_temp / a_temp * 100
    del a_temp, d_temp
    gc.collect()

    # Disconfort Index  #####################################################################
    # https://keisan.casio.jp/exec/system/1202883065

    def disconfort_index(row):
        T = row['air_temperature']
        RH = row['relative_hummd']
        return 0.81 * T + 0.01 * RH * (0.99 * T - 14.3) + 46.3

    df['DI'] = df.apply(disconfort_index, axis=1).astype(np.float32)

    # Apparent Temperature  #####################################################################
    # https://keisan.casio.jp/exec/system/1257417058

    def apparent_temperature(row):
        T = row['air_temperature']
        RH = row['relative_hummd']
        A = 1.76 + 1.4 * row['wind_speed'] ** 0.75
        return 37 - (37 - T) / (0.68 - 0.0014 * RH + 1/A) - 0.29 * T * (1 - RH / 100)

    df['AT'] = df.apply(apparent_temperature, axis=1).astype(np.float32)

    # WCI  #####################################################################
    # https://www.metsoc.jp/tenki/pdf/2010/2010_01_0057.pdf

    def WCI(row):
        T = row['air_temperature']
        U = row['wind_speed']
        return (33 - T) * (10.45 + 10 * U ** 0.5 - U)

    df['WCI'] = df.apply(WCI, axis=1).astype(np.float32)

    # Wind Direction  #####################################################################
    df.loc[df['wind_direction'] == 65535, 'wind_direction'] = np.nan
    df.loc[df['wind_direction'] == 360, 'wind_direction'] = 0
    df['wind_direction'] = np.radians(df['wind_direction'])
    df['wind_direction_sin'] = np.sin(df['wind_direction'])
    df['wind_direction_cos'] = np.cos(df['wind_direction'])

    df['wind_speed_sin'] = df['wind_speed'] * df['wind_direction_sin']
    df['wind_speed_cos'] = df['wind_speed'] * df['wind_direction_cos']

    for c in ['wind_speed_sin', 'wind_speed_cos']:
        df[c] = df[c].astype(np.float16)

    # beaufort_scale  #####################################################################
    beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9),
                (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]

    for item in beaufort:
        df.loc[(df['wind_speed'] >= item[1]) & (df['wind_speed'] < item[2]), 'beaufort_scale'] = item[0]
        
    df['beaufort_scale'] = df['beaufort_scale'].astype(np.float16)

    # Create Features per Site Id  #####################################################################
    for i in range(df['site_id'].nunique()):
        temp = df[df['site_id'] == i]
        temp = temp.sort_values(by='timestamp')
        # Rolling
        cols = ['relative_hummd', 'DI', 'AT']
        for c in cols:
            for window in [24, 36, 48]:
                # Mean
                colname = '{}_roll_{}_mean'.format(c, window)
                temp[colname] = temp[c].rolling(window).mean()
                df.loc[temp.index, colname] = temp.loc[temp.index, colname]
                df[colname] = df[colname].astype(np.float16)
                # Max
                colname = '{}_roll_{}_max'.format(c, window)
                temp[colname] = temp[c].rolling(window).max()
                df.loc[temp.index, colname] = temp.loc[temp.index, colname]
                df[colname] = df[colname].astype(np.float16)
                # Min
                colname = '{}_roll_{}_min'.format(c, window)
                temp[colname] = temp[c].rolling(window).min()
                df.loc[temp.index, colname] = temp.loc[temp.index, colname]
                df[colname] = df[colname].astype(np.float16)

        # Shift
        cols = ['relative_hummd', 'DI', 'AT']
        for c in cols:
            for period in [24, 36, 48]:
                colname = '{}_shift_{}'.format(c, period)
                shifted = temp[c].shift(periods=period)
                temp[colname] = temp[c] - shifted
                df.loc[temp.index, colname] = temp.loc[temp.index, colname]
                df[colname] = df[colname].astype(np.float16)

        del temp
        gc.collect()

    return df


In [9]:
def prepare_data(X, building_data, weather_data, test=False, sampling=None):
    """
    Preparing final dataset with all features.
    """
    
    X = X.merge(building_data, on="building_id", how="left")
    X = X.merge(weather_data, on=["site_id", "timestamp"], how="left")
    
    # Sampling
    if sampling is not None:
        X = X.sample(frac=sampling, random_state=42)
    
    X.timestamp = pd.to_datetime(X.timestamp, format="%Y-%m-%d %H:%M:%S")
    X.square_feet = np.log1p(X.square_feet)
    X['square_feet'] = X['square_feet'].astype(np.float16)
    
    if not test:
        # Data Cleaning
        X['M'] = X['timestamp'].dt.month
        X['D'] = X['timestamp'].dt.day
        X = X.query('not (building_id <= 104 & meter == 0 & M <= 4)')
        X = X.query('not (building_id <= 104 & meter == 0 & M == 5 & D <= 20)')
        X['M'] = X['M'].astype(np.int16)
        del X['D']
        
        X.sort_values("timestamp", inplace=True)
        X.reset_index(drop=True, inplace=True)
    
    gc.collect()
    
    X["hour"] = X.timestamp.dt.hour
    X["weekday"] = X.timestamp.dt.weekday
    
    drop_features = ["timestamp", "sea_level_pressure", "wind_direction", "wind_speed"]

    X.drop(drop_features, axis=1, inplace=True)
    
    # Set dtypes
    categorical_features = ["building_id", "site_id", "meter", "primary_use", "hour", "weekday"]
    for c in categorical_features:
        X[c] = X[c].astype('category')

    if test:
        row_ids = X.row_id
        X.drop("row_id", axis=1, inplace=True)
        return X, row_ids
    else:
        y = np.log1p(X.meter_reading)
        X.drop("meter_reading", axis=1, inplace=True)
        return X, y

In [10]:
%%time
# Data Loading  ############################################
df_train = pd.read_csv(path_train)
df_train = reduce_mem_usage(df_train, use_float16=True)

building = pd.read_csv(path_building)
building = reduce_mem_usage(building, use_float16=True)
le = LabelEncoder()
building.primary_use = le.fit_transform(building.primary_use)

weather_train = pd.read_csv(path_weather_train)
weather_train = reduce_mem_usage(weather_train, use_float16=True)
weather_train = prep_weather_data(weather_train)

# Feature Importance  ############################################
# Prepare X, Y Data
X, y = prepare_data(df_train, building, weather_train, sampling=0.01)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_val, label=y_val)

params = {
    "objective": "regression",
    "boosting": "gbdt",
    "num_leaves": 40,
    "learning_rate": 0.02,
    "feature_fraction": 0.85,
    "reg_lambda": 2,
    "metric": "rmse"
}

model = lgb.train(params, train_set=train_data, num_boost_round=1000, 
                  valid_sets=[train_data, valid_data], verbose_eval=200, early_stopping_rounds=200)

importance_df = pd.DataFrame()
importance_df["feature"] = X.columns.values
importance_df["importance"] = model.feature_importance()
importance_df.sort_values(by='importance', ascending=False, inplace=True)
importance_df.to_csv(f'../Importance/importance_{today}.csv', index=False)

Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 173.90 MB
Decreased by 71.8%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.8%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 2.65 MB
Decreased by 72.4%


/Users/imamuramasataka/Documents/kaggle/ashrae-energy-prediction/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/imamuramasataka/Documents/kaggle/ashrae-energy-prediction/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/imamuramasataka/Documents/kaggle/ashrae-energy-prediction/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 1.07085	valid_1's rmse: 1.13254
[400]	training's rmse: 0.924306	valid_1's rmse: 1.00858
[600]	training's rmse: 0.861908	valid_1's rmse: 0.966927
[800]	training's rmse: 0.82367	valid_1's rmse: 0.948458
[1000]	training's rmse: 0.793181	valid_1's rmse: 0.935697
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 0.793181	valid_1's rmse: 0.935697
CPU times: user 2min 33s, sys: 9.21 s, total: 2min 42s
Wall time: 1min 41s


Create X, Y Data

In [12]:
# Prepare X, y Data  ############################################
# Train Data
print('-' * 20)
print('Create Train Data...')
df_train = pd.read_csv(path_train)
df_train = reduce_mem_usage(df_train, use_float16=True)

building = pd.read_csv(path_building)
building = reduce_mem_usage(building, use_float16=True)
le = LabelEncoder()
building.primary_use = le.fit_transform(building.primary_use)

weather_train = pd.read_csv(path_weather_train)
weather_train = reduce_mem_usage(weather_train, use_float16=True)
weather_train = prep_weather_data(weather_train)

X_train, y_train = prepare_data(df_train, building, weather_train)

for c in X_train.select_dtypes(np.float64).columns.tolist():
    X_train[c] = X_train[c].astype(np.float32)
gc.collect()

with open(f'../input/prep_train_{today}.pkl', 'wb') as f:
    pickle.dump((X_train, y_train), f, protocol=4)
    
del df_train, weather_train, X_train, y_train
gc.collect()
    
# Test Data
print('-' * 20)
print('Create TestData...')
df_test = pd.read_csv(path_test)
weather_test = pd.read_csv(path_weather_test)

df_test = reduce_mem_usage(df_test, use_float16=True)
weather_test = reduce_mem_usage(weather_test, use_float16=True)
weather_test = prep_weather_data(weather_test)

X_test, row_ids = prepare_data(df_test, building, weather_test, test=True)

for c in X_test.select_dtypes(np.float64).columns.tolist():
    X_test[c] = X_test[c].astype(np.float32)
gc.collect()

with open(f'../input/prep_test_{today}.pkl', 'wb') as f:
    pickle.dump((X_test, row_ids), f, protocol=4)
    
print('DONE')

Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 173.90 MB
Decreased by 71.8%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.8%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 2.65 MB
Decreased by 72.4%


/Users/imamuramasataka/Documents/kaggle/ashrae-energy-prediction/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/imamuramasataka/Documents/kaggle/ashrae-energy-prediction/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/imamuramasataka/Documents/kaggle/ashrae-energy-prediction/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 358.65 MB
Decreased by 71.8%
Memory usage of dataframe is 19.04 MB
Memory usage after optimization is: 5.25 MB
Decreased by 72.4%
